# Bayesian Linear Regression Tutorial
This notebook follows the [Bishop](https://www.amazon.com/Pattern-Recognition-Learning-Information-Statistics/dp/0387310738/ref=sr_1_1?ie=UTF8&qid=1474908936&sr=8-1&keywords=pattern+recognition+and+machine+learning) treatment of the Bayesian approach to linear regression.

<img src="1.png">

In [ ]:
import numpy as np
from numpy.random import normal, uniform
from scipy.stats import multivariate_normal as mv_norm
import matplotlib.pyplot as plt
%matplotlib inline

Here we code up our real function

In [ ]:
def real_function(a_0, a_1, noise_sigma, x):
    """
    Evaluates the real function
    """
    N = len(x)
    if noise_sigma==0:
        # Recovers the true function
        return a_0 + a_1*x
    else:
        return a_0 + a_1*x + normal(0, noise_sigma, N)

We now define a class that will perform our parameter updates according to our equations above.  It also handles some useful plotting functionality we'll leverage below.  Note that the only inputs are the prior's mean, covariance and the assumed known noise prediction parameter $\beta$.

In [ ]:
class LinearBayes(object):
    """
    A class that holds parameter prior/posterior and handles 
    the hyper-parameter updates with new data
    
    Note:  variables starting with "v_" indicate Nx1 dimensional 
        column vectors, those starting with "m_" indicate 
        matrices, and those starting with "a_" indicate 
        1xN dimensional arrays.
    
    Args:
        a_m0 (np.array): prior mean vector of size 1xM
        m_S0 (np.ndarray): prior covariance matrix of size MxM
        beta (float): known real-data noise precision
        
    """
    def __init__(self, a_m0, m_S0, beta):
        self.prior = mv_norm(mean=a_m0, cov=m_S0)
        self.v_m0 = a_m0.reshape(a_m0.shape + (1,)) #reshape to column vector
        self.m_S0 = m_S0
        self.beta = beta
        
        self.v_mN = self.v_m0
        self.m_SN = self.m_S0
        self.posterior = self.prior
           
    def get_phi(self, a_x):
        """
        Returns the design matrix of size (NxM) for a feature vector v_x.
        In this case, this function merely adds the phi_0 dummy basis
        that's equal to 1 for all elements.
        
        Args:
            a_x (np.array): input features of size 1xN
        """
        m_phi = np.ones((len(a_x), 2))
        m_phi[:, 1] = a_x
        return m_phi
        
    def set_posterior(self, a_x, a_t):
        """
        Updates mN and SN given vectors of x-values and t-values
        """
        # Need to convert v_t from an array into a column vector
        # to correctly compute matrix multiplication
        v_t = a_t.reshape(a_t.shape + (1,))

        m_phi = self.get_phi(a_x)
        
        self.m_SN = np.linalg.inv(np.linalg.inv(self.m_S0) + self.beta*m_phi.T.dot(m_phi))
        self.v_mN = self.m_SN.dot(np.linalg.inv(self.m_S0).dot(self.v_m0) + \
                                      self.beta*m_phi.T.dot(v_t))
        
        self.posterior = mv_norm(mean=self.v_mN.flatten(), cov=self.m_SN)

    
    def prediction_limit(self, a_x, stdevs):
        """
        Calculates the limit that's "stdevs" standard deviations
        away from the mean at a given value of x.
        
        Args:
            a_x (np.array): x-axis values of size 1xN
            stdevs (float): Number of standard deviations away from
                the mean to calculate the prediction limit
        
        Returns:
            np.array: the prediction limit "stdevs" standard deviations
                away from the mean corresponding to x-values in "v_x"
        
        """
        N = len(a_x)
        m_x = self.get_phi(a_x).T.reshape((2, 1, N))
        
        predictions = []
        for idx in range(N):
            x = m_x[:,:,idx]
            sig_sq_x = 1/self.beta + x.T.dot(self.m_SN.dot(x))
            mean_x = self.v_mN.T.dot(x)
            predictions.append((mean_x+stdevs*np.sqrt(sig_sq_x)).flatten())
        return np.concatenate(predictions)
    
    def generate_data(self, a_x):
        N = len(a_x)
        m_x = self.get_phi(a_x).T.reshape((2, 1, N))
        
        predictions = []
        for idx in range(N):
            x = m_x[:,:,idx]
            sig_sq_x = 1/self.beta + x.T.dot(self.m_SN.dot(x))
            mean_x = self.v_mN.T.dot(x)
            predictions.append(normal(mean_x.flatten(), np.sqrt(sig_sq_x)))
        return np.array(predictions)
    
    def make_contour(self, a_x, a_y, real_parms=[], N=0):
        """
        A helper function to generate contour plots of our probability distribution
        """
        pos = np.empty(a_x.shape + (2,))
        pos[:, :, 0] = a_x
        pos[:, :, 1] = a_y

        plt.contourf(a_x, a_y, self.posterior.pdf(pos), 20)
        plt.xlabel('$w_0$', fontsize=16)
        plt.ylabel('$w_1$', fontsize=16)
        
        if real_parms:
            plt.scatter(real_parms[0], real_parms[1], marker='+', c='black', s=60)
            
        _ = plt.title('Distribution for Weight Parameters using %d datapoint(s)' % N, fontsize=10)
    
    def make_scatter(self, a_x, a_t, real_parms, samples=None, stdevs=None):
        """
        A helper function to plot noisey data, the true function, 
        and optionally a set of lines specified by the nested array of
        weights of size NxM where N is number of lines, M is 2 for 
        this simple model
        """
        plt.scatter(a_x, a_t, alpha=0.5)
        plt.xlabel('x')
        plt.ylabel('t')

        plt.plot([-1, 1], real_function(real_parms[0], real_parms[1], 0, np.array([-1., 1.])), 'r')

        _ = plt.title('Real Data from Noisey Linear Function')
        
        if samples:
            weights = self.posterior.rvs(samples)
            for weight in weights: 
                plt.plot([-1, 1], real_function(weight[0], weight[1], 0, np.array([-1., 1.])), 'black')
                _ = plt.title('Lines Sampled from Posterior Distribution vs Real Line and Data')
                
        if stdevs:
            a_xrange = np.linspace(-1, 1, 100)
            y_upper = self.prediction_limit(a_xrange, stdevs)
            y_lower = self.prediction_limit(a_xrange, -stdevs)
            plt.plot(a_xrange, y_upper, '+', c='green', linewidth=4.0)
            plt.plot(a_xrange, y_lower, '+', c='green', linewidth=4.0)
            _ = plt.title('Lines Sampled from Posterior Distribution vs Real Line and Data')

First, let's set our real function parameters for.  These are what we're trying to recover.  We also set a value for the noise precision $\beta$, which we assume is known to us.  We then generate some noisey data from this true function and noise level.

In [ ]:
# Real function parameters
a_0 = -0.3
a_1 = 0.5
noise_sigma = 0.2
beta = 1/noise_sigma**2
# Generate input features from uniform distribution
np.random.seed(20) # Set the seed so we can get reproducible results
x_real = uniform(-1, 1, 1000)
# Evaluate the real function for training example inputs
t_real = real_function(a_0, a_1, noise_sigma, x_real)

Now let's set up our prior distribution.  We need to assign hyper-parameters that represent the prior's mean and covariance.  We said above that we'll assume an isotropic gaussian, which means the covariance will be constant and the same in all directions.  Let's choose a precision of $\alpha = 2.0$.  Our covariance is then the inverse of this value multiplied by the identity matrix.  We then instantiate an instance of our helper `LinearBayes` class defined above.

In [ ]:
alpha = 2.0
v_m0 = np.array([0., 0.])
m_S0 = 1/alpha*np.identity(2)

linbayes = LinearBayes(v_m0, m_S0, beta)

Our real, noisey data is shown below as the blue dots.  The "true" function parameters we're trying to find are represented by the red line function.

In [ ]:
linbayes.make_scatter(x_real, t_real, real_parms = [a_0, a_1])

Let's take a look at the prior distribution.  The real values for the true parameters ($a_0=-0.3, a_1=0.5$) are represented as a black tick mark.  Note that our prior distribution is centered at $(0, 0)$ and assigns relatively low, but non-zero probability to where the true parameter values lie.

In [ ]:
x, y = np.mgrid[-1:1:.01, -1:1:.01]
linbayes.make_contour(x, y, real_parms=[a_0, a_1], N=0)

Let's pull out a single data point from our real data and see how it relates to the true function.

In [ ]:
N=1
linbayes.make_scatter(x_real[0:N], t_real[0:N], real_parms=[a_0, a_1])

Now let's use this data point to update our estimation of the red line.  We'll do this by updating the $ \{\vec{m}_N, \vec{S}_N\}$ parameters according to our formulae above, and plot the posterior distribution for the model parameters.

In [ ]:
linbayes.set_posterior(x_real[0:N], t_real[0:N])
linbayes.make_contour(x, y, real_parms=[a_0, a_1], N=N)

Let's pull random samples from this distribution to get a collection of $\{w_0, w_1\}$.  These are 5 different estimates for our line.  We'll then plot what those lines look like in comparison to both the true model and our single data point used in the parameter update.

In [ ]:
linbayes.make_scatter(x_real[0:N], t_real[0:N], real_parms=[a_0, a_1], samples=5)

Note that these lines tend to come close to the data point but vary a great deal far from the point.  

But generally, we're not interested in what the values of $\{w_0, w_1\}$ are for their own sake.  We only care about them so that we can make predictions of what $t$ should be given a new $\vec{x}$.  So instead of looking at particular lines by sampling $\{w_0, w_1\}$ from our posterior distribution, let's look at the **predictive distribution** itself by plugging our posterior distribution into $(3)$.

Written with all of our parameters:

$$ p(t \mid \vec{t}, \alpha, \beta) = \int p(t \mid \vec{w}, \beta)p(\vec{w} \mid \vec{t}, \alpha, \beta) d\vec{w}  $$


where $p(t \mid \vec{w}, \beta)$ represents our model $(2)$ for a set of parameters and $p(\vec{w} \mid \vec{t}, \alpha, \beta)$ is our posterior distribution we've been calculating.  This evaluates analytically to be:
$$ p(t \mid \vec{t}, \alpha, \beta) = Norm(t \mid \vec{m}_N^T\vec{\phi}(\vec{x}), \sigma_N^2(\vec{x})) \tag{1}$$ where 
$$ \sigma_N^2(\vec{x}) = \frac{1}{\beta} + \vec{\phi}(\vec{x})^T\vec{S}_N\vec{\phi}(\vec{x}) \tag{2}$$

This is our ultimate prediction engine we wanted.  First of all, notice that the mean and variance of this normal distribution **depend on $\vec{x}$**.  This means we have a sort of sliding normal distribution that changes it's mean and variance as it goes from left to right.  That's hard to visualize, so let's take a look at the curves that are $\pm \sigma_N$ from its mean.  This represents confidence intervals around our estimate for what $t$ should be at a given value of $x$.

In [ ]:
linbayes.make_scatter(x_real[0:N], t_real[0:N], real_parms=[a_0, a_1], stdevs=1)

Note that these upper/lower bounds are closest together around the one data point we used.

Now let's try adding a second data point.

In [ ]:
N=2
linbayes.make_scatter(x_real[0:N], t_real[0:N], real_parms=[a_0, a_1])

Here's the posterior distribution for the weights using both data points.

In [ ]:
linbayes.set_posterior(x_real[0:N], t_real[0:N])
linbayes.make_contour(x, y, real_parms=[a_0, a_1], N=N)

Let's pull some samples from this distribution to generate some example estimates and include the 1-standard deviation confidence intervals.

In [ ]:
linbayes.make_scatter(x_real[0:N], t_real[0:N], real_parms=[a_0, a_1], samples=5, stdevs=1)

Now let's use 10 points

In [ ]:
N=10
linbayes.make_scatter(x_real[0:N], t_real[0:N], real_parms=[a_0, a_1])

In [ ]:
linbayes.set_posterior(x_real[0:N], t_real[0:N])
linbayes.make_contour(x, y, real_parms=[a_0, a_1], N=N)

In [ ]:
linbayes.make_scatter(x_real[0:N], t_real[0:N], real_parms=[a_0, a_1], samples=5, stdevs=1)

Let's use the whole data set (1000 points).

In [ ]:
N=1000
linbayes.make_scatter(x_real[0:N], t_real[0:N], real_parms=[a_0, a_1])

In [ ]:
linbayes.set_posterior(x_real[0:N], t_real[0:N])
linbayes.make_contour(x, y, real_parms=[a_0, a_1], N=N)

In [ ]:
linbayes.make_scatter(x_real[0:N], t_real[0:N], real_parms=[a_0, a_1], samples=5, stdevs=1)

Note that as the sample size increased, the posterior distribution became much sharper, which expresses greater confidence in the parameter estimates.  When 1000 points were used, sampling from the posterior weights distribution results in lines that are essentially on top of the true line.

However, if we have such strong confidence (as seen by a very sharp posterior distribution) why are our $\pm \sigma_N$ lines so far away still?  All of our sampled lines are so precise and accurate that we can't even *see* the true, red line in the above graph.

This is because there's a limit to our confidence, and that limit is set by the amount of noise in the data.  Let's inspect the predictive distribution's variance:
$$ \sigma_N^2(\vec{x}) = \frac{1}{\beta} + \vec{\phi}(\vec{x})^T\vec{S}_N\vec{\phi}(\vec{x})$$

There are two terms:  the first is the variance of the gaussian noise intrinsic in the data and the second depends on our posterior's covariance $\vec{S}_N$.  This shows that even if our posterior covariance shrinks to zero, we still have the noise in our data limiting our estimation.  Below we plot our confidence intervals along with $t_{real} \pm \sigma_{noise}$

In [ ]:
a_x = np.linspace(-1, 1, 100)
plt.plot(a_x, real_function(a_0 + noise_sigma, a_1, 0, a_x), 'x', c='r', linewidth=1.0)
plt.plot(a_x, real_function(a_0 - noise_sigma, a_1, 0, a_x), 'x', c='r', linewidth=1.0)
linbayes.make_scatter([], [], real_parms=[a_0, a_1], stdevs=1)
_ = plt.title('Noise and Parameter Confidence Around True Function')

Note that we have essentially acheived our goal:  we have recovered a probability distribution that not only gives a point estimate of the "true" model, but accounts for noise in both the data and the parameter estimates.

As a final step, let's generate some data from this probability distribution (red) and plot it along real data and the "true" model.

In [ ]:
a_x = np.linspace(-1, 1, 1000)
linbayes.make_scatter(x_real[0:N], t_real[0:N], real_parms=[a_0, a_1])
_ = plt.scatter(a_x, linbayes.generate_data(a_x), c='r', alpha=0.5)